In [2]:
from transformers import BertModel
import torch as t
from transformers import BertTokenizer
from torch import nn
import onnx

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
class my_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(10, 3)
        self.l2 = nn.Linear(20, 3)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
    
    def forward(self, input1, input2):
        output1 = self.l1(input1)
        # output = self.dropout(output)
        # output = self.relu(output)
        output2 = self.l2(input2)
        output2 = output2.transpose(0,1)
        return t.matmul(output1,output2)

In [13]:
bt = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

sent = "I love you very much."
input_ids = t.tensor([bt.encode(sent)])
am = t.ones(input_ids.size())

In [7]:
model(input_ids, am)[0].size()

torch.Size([1, 6, 768])

In [3]:
model = my_model()
x1 = t.randn(2,10)
x2 = t.randn(3,20)
model(x1, x2)

tensor([[-0.2710, -0.2388, -0.2480],
        [ 0.7592,  0.7918,  0.0612]], grad_fn=<MmBackward>)

In [15]:
dynamic_axes={'input_idx':{0:'bs', 1:'seq_len'}, 'attention_mask':{0:'bs', 1:'seq_len'}, 'output':{0:'bs', 1:'seq_len', 2:'hid_dim'}}
with t.no_grad():
    t.onnx.export(
        model, 
        (input_ids, am),
        "model.onnx", 
        opset_version=10, 
        input_names=['input_idx', 'attention_mask'], 
        output_names=['output'],
        dynamic_axes=dynamic_axes)

In [5]:
onnx_model = onnx.load("test.onnx")

In [6]:
try:
    onnx.checker.check_model(onnx_model)
except Exception:
    print("incorrect")
else:
    print("correct")

correct
